<a href="https://colab.research.google.com/github/jaivanti/syracuse/blob/main/espcn_finalycbccr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import os
import math
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
from IPython.display import display
from keras.models import load_model
import PIL

In [ ]:
def get_lowres_image(img, upscale_factor):
  """Return low-resolution image to use as model input."""
  return img.resize((img.size[0] // upscale_factor, img.size[1] // upscale_factor),PIL.Image.BICUBIC)

In [ ]:

def upscale_image(model, img):
    """Predict the result based on input image and restore the image as RGB."""
    ycbcr = img.convert("YCbCr")
    y, cb, cr = ycbcr.split()
    y = img_to_array(y)
    y = y.astype("float32") / 255.0

    input = np.expand_dims(y, axis=0)
    out = model.predict(input)


    out_img_y = out[0]
    out_img_y *= 255.0

    # Restore the image in RGB color space.
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = out_img_y.reshape((np.shape(out_img_y)[0], np.shape(out_img_y)[1]))
    out_img_y = PIL.Image.fromarray(np.uint8(out_img_y), mode="L")
    out_img_cb = cb.resize(out_img_y.size, PIL.Image.BICUBIC)
    out_img_cr = cr.resize(out_img_y.size, PIL.Image.BICUBIC)
    out_img = PIL.Image.merge("YCbCr", (out_img_y, out_img_cb, out_img_cr)).convert(
        "RGB"
    )
    return out_img


In [ ]:
!ffmpeg -s 352x288 -i /content/bus_cif.yuv image%05d.bmp

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
start = time.time()
img = load_img('/content/image00003.bmp')
ycbcr = img.convert("YCbCr")
y, cb, cr = ycbcr.split()
y = img_to_array(y)
y = y.astype("float32") / 255.0
input = np.expand_dims(y, axis=0)
end = time.time()
t1 = end-start
print("t1 is ", t1)
st = time.time()
out = model.predict(input)
en = time.time()
t3 = en-st
print("upscaling inf time is ",t3 )
start1 = time.time()
out_img_y = out[0]
out_img_y *= 255.0

# Restore the image in RGB color space.
out_img_y = out_img_y.clip(0, 255)
out_img_y = out_img_y.reshape((np.shape(out_img_y)[0], np.shape(out_img_y)[1]))
out_img_y = PIL.Image.fromarray(np.uint8(out_img_y), mode="L")
out_img_cb = cb.resize(out_img_y.size, PIL.Image.BICUBIC)
out_img_cr = cr.resize(out_img_y.size, PIL.Image.BICUBIC)
out_img = PIL.Image.merge("YCbCr", (out_img_y, out_img_cb, out_img_cr)).convert(
    "RGB"
)
end1 = time.time()
t2 = end1-start1
print("t2 is ", t2)
print("t1 + t2 is ", t1 + t2)
print("whole is", t1 + t2 + t3)





t1 is  0.023955583572387695


NameError: ignored

In [ ]:
!ffmpeg -i /content/drive/MyDrive/SRCDN/akiyo_cif.y4m /content/drive/MyDrive/SRCDN/test.yuv

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
#model = load_model("/content/drive/MyDrive/SRCDN/model/network.h5")   #suyash
model = load_model("/content/drive/MyDrive/SRCDN/model/gfgModel.h5")   #jaivanti

In [ ]:
#!mkdir /content/drive/MyDrive/SRCDN/LR/video2
!mkdir /content/drive/MyDrive/SRCDN/LR/video1


In [ ]:
!mkdir /content/drive/MyDrive/SRCDN/result/video11

In [ ]:
%cd /content/drive/MyDrive/SRCDN/LR/video10
!ffmpeg  -i /content/drive/MyDrive/SRCDN/1080p_videos/short_videos/bright.mp4 %d.png

/content/drive/.shortcut-targets-by-id/1lZzJclOmbbPx_0_m3KHq80BRx5R6ZSuO/SRCDN/LR/video10
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
im = load_img('/content/drive/MyDrive/SRCDN/result/video10/op1.png')
im1 = get_lowres_image(im, 4)


In [ ]:
im.size

(2560, 1440)

In [ ]:
#/content/drive/MyDrive/SRCDN/LR/video1/1.png
upscale_factor = 3
t_sum = 0
for i in range(1, 17):
  img_path = ('/content/drive/MyDrive/SRCDN/LR/video10/' + str(i) + '.png')
  img = load_img(img_path)
  lowres_input = get_lowres_image(img, upscale_factor)
  w = lowres_input.size[0] * upscale_factor
  h = lowres_input.size[1] * upscale_factor
  #highres_img = img.resize((w, h))
  start = time.time()
  prediction = upscale_image(model, lowres_input)
  end = time.time()
  print(end-start)
  dif = end-start
  t_sum += dif
  prediction.save('/content/drive/MyDrive/SRCDN/result/video10/op' + str(i) + '.png')
  
avg = t_sum/16
print("the avg is", avg)



0.177321195602417
0.18551135063171387
0.18527579307556152
0.17645525932312012
0.1679825782775879
0.17114925384521484
0.18181324005126953
0.1724405288696289
0.17189788818359375
0.17500829696655273
0.17695856094360352
0.23647403717041016
0.17426085472106934
0.19145774841308594
0.27732419967651367


FileNotFoundError: ignored

In [ ]:
print("asd", 25)

asd 25


In [ ]:
type(prediction)

PIL.Image.Image

In [ ]:
prediction.save('/content/drive/MyDrive/SRCDN/result/op.png')

In [ ]:

!ffmpeg -f image2 -framerate 30 -i /content/drive/MyDrive/SRCDN/result/video6/op%d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p flame10809sr.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib